In [37]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import shutil

# # I have taken 350 images of cats and 350 images of dogs

In [12]:
# Function to Extract features from the images
def image_feature(direc):
    model = InceptionV3(weights='imagenet', include_top=False)
    features = [];
    img_name = [];
    for i in tqdm(direc):
        fname='cluster'+'/'+i
        img=image.load_img(fname,target_size=(224,224))
        x = img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        feat=model.predict(x)
        feat=feat.flatten()
        features.append(feat)
        img_name.append(i)
    return features,img_name

In [2]:
img_path=os.listdir('cluster')

In [13]:
img_features,img_name=image_feature(img_path)

100%|████████████████████████████████████████| 700/700 [02:15<00:00,  5.17it/s]


In [50]:
image_cluster = pd.DataFrame(img_name,columns=['image'])

In [51]:
image_cluster

,image
0,cat.0.jpg
1,cat.1.jpg
2,cat.10.jpg
3,cat.100.jpg
4,cat.101.jpg
...,...
695,dog.95.jpg
696,dog.96.jpg
697,dog.97.jpg
698,dog.98.jpg


In [20]:
#Creating Clusters
k = 2
clusters = KMeans(k, random_state = 40)
clusters.fit(img_features)

KMeans(n_clusters=2, random_state=40)

In [52]:
image_cluster["clusterid"] = clusters.labels_ # To mention which image belong to which cluster

In [53]:
image_cluster # 0 denotes cat and 1 denotes dog

,image,clusterid
0,cat.0.jpg,0
1,cat.1.jpg,0
2,cat.10.jpg,0
3,cat.100.jpg,0
4,cat.101.jpg,0
...,...,...
695,dog.95.jpg,1
696,dog.96.jpg,1
697,dog.97.jpg,1
698,dog.98.jpg,1


In [31]:
image[image.clusterid == 0] # There are only 321 images in cats which means rest 29 images of cats are misclassified in dogs cluster

,image,clusterid
0,cat.0.jpg,0
1,cat.1.jpg,0
2,cat.10.jpg,0
3,cat.100.jpg,0
4,cat.101.jpg,0
...,...,...
344,cat.94.jpg,0
345,cat.95.jpg,0
347,cat.97.jpg,0
348,cat.98.jpg,0


In [35]:
# Made folder to seperate images
os.mkdir('cats')
os.mkdir('dogs')

In [56]:
# Images will be seperated according to cluster they belong
for i in range(len(image_cluster)):
    if image_cluster['clusterid'][i]==0:
        shutil.move(os.path.join('cluster', image_cluster['image'][i]), 'cats')
    else:
        shutil.move(os.path.join('cluster', image_cluster['image'][i]), 'dogs')